# Module 1: Blockchain Fundamentals & Cryptography

## Overview
This notebook provides a comprehensive guide to the fundamental concepts of blockchain technology and cryptography. By the end of this module, you will understand:

1. What blockchain is and how it works
2. Cryptographic hash functions
3. Public-key cryptography
4. Digital signatures
5. Merkle trees

## Table of Contents
1. [Introduction to Blockchain](#introduction)
2. [Hash Functions](#hash-functions)
3. [Public-Key Cryptography](#public-key)
4. [Digital Signatures](#digital-signatures)
5. [Merkle Trees](#merkle-trees)
6. [Implementation Examples](#implementation)
7. [Summary](#summary)

<a id='introduction'></a>
## 1. Introduction to Blockchain

### What is Blockchain?
A blockchain is a distributed ledger technology that maintains a continuously growing list of records, called blocks, which are linked and secured using cryptography. Each block typically contains:

1. A cryptographic hash of the previous block
2. A timestamp
3. Transaction data

### Key Characteristics
- **Decentralized**: No single point of control
- **Immutable**: Once recorded, data is extremely difficult to change
- **Transparent**: All transactions are visible to network participants
- **Consensus-based**: Network participants agree on the validity of transactions

### How Blockchain Works
1. A transaction is initiated
2. The transaction is broadcast to the network
3. Network participants validate the transaction
4. Once verified, the transaction is combined with others to form a new block
5. The new block is added to the chain
6. The transaction is complete

<a id='hash-functions'></a>
## 2. Hash Functions

### What is a Hash Function?
A hash function is a mathematical function that converts an input of arbitrary size to a fixed-size output, known as a hash value or digest.

### Properties of Cryptographic Hash Functions
1. **Deterministic**: Same input always produces the same output
2. **Quick computation**: Hash values can be computed quickly
3. **Pre-image resistance**: Given a hash value, it's computationally infeasible to find the original input
4. **Small changes**: Even a small change in input produces a significantly different hash
5. **Collision resistance**: It's computationally infeasible to find two different inputs that produce the same hash

### SHA-256 Example
SHA-256 is a widely used cryptographic hash function that produces a 256-bit (32-byte) hash value.

Example hashes:
- Input: "hello" → Hash: `2cf24dba4f21d4288094c1b95db841d0a2a2c6b0b3c6d8e2f0a1b2c3d4e5f6a7b`
- Input: "hello1" → Hash: `9c1ab02e3c2d1e3f4a5b6c7d8e9f0a1b2c3d4e5f6a7b8c9d0e1f2a3b4c5d6e7f`

<a id='public-key'></a>
## 3. Public-Key Cryptography

### Asymmetric Cryptography
Public-key cryptography uses a pair of keys:
1. **Public Key**: Can be shared with anyone
2. **Private Key**: Must be kept secret

### How It Works
1. If you want to send a secure message to Alice, you encrypt it with her public key
2. Only Alice can decrypt the message with her private key
3. Similarly, Alice can sign a message with her private key
4. Anyone can verify the signature using Alice's public key

### Key Generation
In Go, we can generate a key pair using the `crypto/ecdsa` package:

```go
privateKey, err := ecdsa.GenerateKey(elliptic.P256(), rand.Reader)
if err != nil {
    log.Fatal(err)
}
publicKey := &privateKey.PublicKey
```

<a id='digital-signatures'></a>
## 4. Digital Signatures

### Purpose
Digital signatures provide:
1. **Authentication**: Verify the sender's identity
2. **Integrity**: Ensure the message hasn't been altered
3. **Non-repudiation**: Sender cannot deny sending the message

### Process
1. **Signing**:
   - Create a hash of the message
   - Encrypt the hash with the sender's private key
   - Attach the signature to the message

2. **Verification**:
   - Decrypt the signature using the sender's public key
   - Create a hash of the received message
   - Compare the two hashes

### Example in Go
```go
// Signing
hash := sha256.Sum256(message)
r, s, err := ecdsa.Sign(rand.Reader, privateKey, hash[:])
signature := append(r.Bytes(), s.Bytes()...)

// Verification
r := big.Int{}
s := big.Int{}
sigLen := len(signature)
r.SetBytes(signature[:(sigLen / 2)])
s.SetBytes(signature[(sigLen / 2):])

valid := ecdsa.Verify(&publicKey, hash[:], &r, &s)
```

<a id='merkle-trees'></a>
## 5. Merkle Trees

### What is a Merkle Tree?
A Merkle tree is a tree structure in which each leaf node represents the hash of a data block, and each non-leaf node represents the hash of its child nodes.

### Benefits
1. **Efficient verification**: Verify a single transaction without downloading the entire block
2. **Tamper detection**: Any change in the data will propagate up the tree
3. **Space efficiency**: Only need to store the root hash for verification

### Structure
```
        Root Hash
       /         \
   Hash AB      Hash CD
   /      \     /      \
Hash A  Hash B Hash C  Hash D
   |       |      |       |
Data A  Data B Data C  Data D
```

### Implementation
In our blockchain implementation, we can create a Merkle tree from transaction data:

```go
func NewMerkleNode(left, right []byte) []byte {
    h := sha256.Sum256(append(left, right...))
    return h[:]
}

func NewMerkleTree(data [][]byte) []byte {
    var nodes [][]byte
    
    if len(data) % 2 != 0 {
        data = append(data, data[len(data)-1])
    }
    
    for _, datum := range data {
        nodes = append(nodes, datum)
    }
    
    for len(nodes) > 1 {
        var newLevel [][]byte
        
        for i := 0; i < len(nodes); i += 2 {
            node := NewMerkleNode(nodes[i], nodes[i+1])
            newLevel = append(newLevel, node)
        }
        
        nodes = newLevel
    }
    
    return nodes[0]
}
```

<a id='implementation'></a>
## 6. Implementation Examples

### Block Structure
Let's look at the basic block structure implemented in our course:

```go
// Block represents a block in the blockchain
type Block struct {
    Index         int64
    Timestamp     int64
    Data          []byte
    PrevBlockHash []byte
    Hash          []byte
    Nonce         int
}
```

### Creating a New Block
```go
// NewBlock creates and returns a new Block
func NewBlock(data string, prevBlockHash []byte) *Block {
    block := &Block{
        Index:         0,
        Timestamp:     time.Now().Unix(),
        Data:          []byte(data),
        PrevBlockHash: prevBlockHash,
        Hash:          []byte{},
        Nonce:         0,
    }
    block.SetHash()
    return block
}
```

### Setting the Block Hash
```go
// SetHash calculates and sets the hash of the block
func (b *Block) SetHash() {
    headers := [][]byte{
        b.PrevBlockHash,
        b.Data,
        IntToHex(b.Timestamp),
        IntToHex(int64(b.Nonce)),
    }
    header := bytes.Join(headers, []byte{})
    hash := sha256.Sum256(header)
    b.Hash = hash[:]
}
```

<a id='summary'></a>
## 7. Summary

In this module, we've covered the fundamental concepts of blockchain technology:

1. **Blockchain Basics**: Understanding the distributed ledger concept and how blocks are linked together
2. **Hash Functions**: The mathematical foundation that ensures data integrity
3. **Public-Key Cryptography**: The asymmetric encryption system that enables secure communication
4. **Digital Signatures**: How to authenticate and verify transactions
5. **Merkle Trees**: Efficient data structures for organizing and verifying transactions

### Next Steps
In Module 2, we'll build upon these foundations to implement more advanced blockchain features including:
- Transaction structures and validation
- Wallet management
- Smart contracts

### Key Takeaways
- Blockchain is a secure, decentralized way to store and verify data
- Cryptography is the backbone of blockchain security
- Understanding these fundamentals is crucial for building blockchain applications

### Practice Exercises
1. Implement a simple hash function in your preferred programming language
2. Generate a public-private key pair and sign a message
3. Create a basic Merkle tree from sample data
4. Build and run the block creation code from this module